Выбор большой языковой модели

In [ ]:
from llamba.chatmodels.ollama import OllamaModel
chatbot = OllamaModel(url="http://127.0.0.1:11434/", endpoint="api/generate", model="llama3", num_threads=1, check_connection_timeout=15, request_timeout=15) # опционально, параметр model может быть иным -- см. поддерживаемые модели Ollama
connection = chatbot.check_connection()
print(connection)

True


Подготовка данных

In [6]:
import torch
from torch import nn
import pandas as pd
import numpy as np

from llamba_library.bioage_model import BioAgeModel
from llamba_library.functions import get_shap_dict
from txai_omics_3.models.tabular.widedeep.ft_transformer import WDFTTransformerModel, FN_SHAP, FN_CHECKPOINT


#### Данные

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

data = pd.DataFrame(my_data, index=[0])


#### Модель
    
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(fn_model)
bioage_model = BioAgeModel(model=model)

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
shap_dict = get_shap_dict(FN_SHAP)
explainer = shap_dict['explainer']
feats = data.drop(['Age'], axis=1).columns.to_list()

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint H:\Lobachevsky\txai_omics_3\data\immuno\model.ckpt`
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


Составление запроса для экспертной системы 

In [7]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)
prompts = connector.generate_prompts(n=2, data=data, feats=feats, explainer=explainer) # n - количество признаков, внесших наибольший вклад
print(prompts)

['What is CXCL9? What does an increased level of CXCL9 mean?', 'What is CD40LG? What does an increased level of CD40LG mean?']


Передача экспертной системе данных 

In [8]:
res = connector.query_prompts()
print(res)

CXCL9: 2599.629474
CD40LG: 1853.847406
CXCL9 is a chemokine, specifically a CXC chemokine, that plays a role in immune responses. It is also known as monokine induced by gamma-interferon (MIG). 

An increased level of CXCL9 has been associated with various conditions, including inflammatory disorders such as rheumatoid arthritis and multiple sclerosis, as well as certain types of cancer. In the context of gerontology, elevated CXCL9 levels may be indicative of chronic inflammation and immune activation in older adults, which can contribute to age-related diseases like atherosclerosis, osteoporosis, and frailty.

CD40LG, also known as TNFSF5 or TRAF2, is a protein that plays a crucial role in immune system function. It is a ligand for the receptor CD40, which is primarily expressed on B cells and certain antigen-presenting cells.

An increased level of CD40LG has been associated with various autoimmune disorders, such as lupus and rheumatoid arthritis, where it contributes to aberrant B